In [2]:
!pip install music21


[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import xml.etree.ElementTree as ET
import warnings
import music21


from music21 import *
from music21.musicxml.xmlObjects import MusicXMLImportException


class MusicXMLWarning(UserWarning):
    pass


class OttomanMusicPartParser(musicxml.xmlToM21.PartParser):
    def xmlMeasureToMeasure(self, mxMeasure: ET.Element) -> stream.Measure:        
        measureParser = OttomanMusicMeasureParser(mxMeasure, parent=self)
        # noinspection PyBroadException
        try:
            measureParser.parse()
        except MusicXMLImportException as e:
            e.measureNumber = str(measureParser.measureNumber)
            e.partName = self.stream.partName
            raise e
        except Exception as e:  # pylint: disable=broad-exception-caught
            warnings.warn(
                f'The following exception took place in m. {measureParser.measureNumber} in '
                + f'part {self.stream.partName}.',
                MusicXMLWarning
            )
            raise e

        self.lastMeasureParser = measureParser

        if measureParser.staves > self.maxStaves:
            self.maxStaves = measureParser.staves

        if measureParser.transposition is not None:
            self.updateTransposition(measureParser.transposition)

        self.firstMeasureParsed = True
        self.staffReferenceList.append(measureParser.staffReference)

        m = measureParser.stream
        self.setLastMeasureInfo(m)
        # TODO: move this into the measure parsing,
        #     because it should happen on a voice level.
        if measureParser.fullMeasureRest is True:
            # recurse is necessary because it could be in voices...
            r1 = m[note.Rest].first()

            if t.TYPE_CHECKING:
                # fullMeasureRest is True, means Rest will be found
                assert r1 is not None

            if self.lastTimeSignature is not None:
                lastTSQl = self.lastTimeSignature.barDuration.quarterLength
            else:
                lastTSQl = 4.0  # sensible default.

            if (r1.fullMeasure is True  # set by xml measure='yes'
                                    or (r1.duration.quarterLength != lastTSQl
                                        and r1.duration.type in ('whole', 'breve')
                                        and r1.duration.dots == 0
                                        and not r1.duration.tuplets)):
                r1.duration.quarterLength = lastTSQl
                r1.fullMeasure = True

        # NB: not coreInsert, because barDurationProportion()
        # is called in adjustTimeAttributesFromMeasure()
        self.stream.insert(self.lastMeasureOffset, m)
        self.adjustTimeAttributesFromMeasure(m)
        # TODO: musicxml4: listening

        return m


class OttomanMusicXMLImporter(musicxml.xmlToM21.MusicXMLImporter):
    """
    Subclass MusicXMLImporter to allow importing non-standard accidentals
    """
    def xmlPartToPart(self, mxPart, mxScorePart):
        '''
        Given a <part> object and the <score-part> object, parse a complete part.
        '''
        parser = OttomanMusicPartParser(mxPart, mxScorePart=mxScorePart, parent=self)
        parser.parse()
        if parser.appendToScoreAfterParse is True:
            return parser.stream
        else:
            return None


class OttomanMusicMeasureParser(musicxml.xmlToM21.MeasureParser):
    """
    Subclass MusicXMLImporter to allow importing non-standard accidentals
    """
    def nonTraditionalKeySignature(self, mxKey):
        '''
        Returns a KeySignature object that represents a nonTraditional Key Signature
        '''
        # noinspection PyShadowingNames
        allChildren = list(mxKey)

        lastTag = None
        allSteps = []
        allAlters = []
        allAccidentals = []

        for c in allChildren:
            tag = c.tag
            if lastTag == 'key-alter' and tag == 'key-step':
                allAccidentals.append(None)

            if tag == 'key-step':
                allSteps.append(c.text)
            elif tag == 'key-alter':
                allAlters.append(float(c.text))
            elif tag == 'key-accidental':
                allAccidentals.append(c.text)
            lastTag = tag

        if len(allAccidentals) < len(allAlters):
            allAccidentals.append(None)
        if len(allSteps) != len(allAlters):
            raise MusicXMLImportException(
                'For non traditional signatures each step must have an alter')

        ks = key.KeySignature(sharps=None)

        alteredPitches = []
        for i in range(len(allSteps)):
            thisStep = allSteps[i]
            thisAlter = allAlters[i]
            thisAccidental = allAccidentals[i]
            p = pitch.Pitch(thisStep)
            if thisAccidental is not None:
                if thisAccidental in self.mxAccidentalNameToM21:
                    accidentalName = self.mxAccidentalNameToM21[thisAccidental]
                else:
                    accidentalName = thisAccidental
                p.accidental = OttomanMusicAccidental(accidentalName)
                p.accidental.alter = thisAlter
            else:
                p.accidental = OttomanMusicAccidental(thisAlter)

            alteredPitches.append(p)

        ks.alteredPitches = alteredPitches
        return ks


class OttomanMusicAccidental(music21.pitch.Accidental):
    '''
    Subclass Accidental to allow importing non-standard accidentals
    '''
    def set(self, name, *args, allowNonStandardValue=True):
        # noinspection PyShadowingNames
        super().set(name, *args, allowNonStandardValue=allowNonStandardValue)


In [4]:
"""
Parse the Teslim and extract information relevant to inferring the makam.
- input: path to the musicxml for a sarki
- return: a dictionary of the following format:
    {
    "start_note": first note of the Teslim (music21 Note object),
    "end_note": last note of the Teslim (music21 Note),
    "range": (lowest note in the Teslim, highest note of the Teslim) (pair of music21 Notes),
    "accidentals": list of notes with accidentals in the order that they appear in the Teslim (list of music21 Notes)
    }
"""
folder_path = "../With Key Signatures/"

class DoesNotHaveTeslim(Exception):
    def __init__(self):
        self.message = "There does not exist a Teslim in this piece. "
        super().__init__(self.message)

def find_teslim(filename):
    MI = OttomanMusicXMLImporter()
    score = MI.scoreFromFile(folder_path+filename)
    start_measure_num = None
    end_measure_num = None

    for element in score.recurse().getElementsByClass(expressions.TextExpression):
        if element.content.lower() == 'teslim':
            start_measure_num =  element.getContextByClass('Measure').number
        if element.content.lower() == 'hane 2':
            end_measure_num = element.getContextByClass('Measure').number - 1
            break

    teslim = stream.Score()

    if start_measure_num is None:
        raise DoesNotHaveTeslim
    
    # Iterate through the measures in the original score and add them to the new score
    for measure in score.measures(start_measure_num, end_measure_num):
        teslim.insert(measure.offset, measure)

    return teslim

In [39]:
def equal_pitch(pitch1, pitch2):
    return (pitch1.step == pitch2.step 
            and pitch1.octave == pitch2.octave 
            and pitch1.accidental == pitch2.accidental)

def get_unique_pitches(notes):
    pitches = [notes[0].pitch]
    for note in notes[1:]:
        if all(equalPitches(pitch, note.pitch) for pitch in pitches):
            pitches.append(note.pitch)
                
    return pitches

def extract_teslim_info(teslim):
    first_measure = teslim.parts[0].getElementsByClass('Measure')[0]
    start_note = first_measure.getElementsByClass('Note')[0]

    last_measure = teslim.parts[0].getElementsByClass('Measure')[-1]
    end_note = last_measure.getElementsByClass('Note')[-1]
    
    all_notes = teslim.parts[0].recurse().notes
    lowest_note = min(all_notes, key=lambda x: x.pitch)
    highest_note = max(all_notes, key=lambda x: x.pitch)
    
    accidentals = [note for note in all_notes if note.pitch.accidental is not None]
    
    result = {
        "start_note": start_note,
        "end_note": end_note,
        "range": (lowest_note, highest_note),
        "accidentals": accidentals,
        "all_pitches": get_unique_pitches(all_notes)
    }
    return result

In [17]:
def find_maqams(filename):
    maqam_info = {}
    MI = OttomanMusicXMLImporter()
    score = MI.scoreFromFile(folder_path+filename)
    start_measure_num = None
    end_measure_num = None

    for element in score.recurse().getElementsByClass(expressions.TextExpression):
        if start_measure_num is None:
            start_measure_num =  element.getContextByClass('Measure').number
        elif end_measure_num is None:
            end_measure_num = element.getContextByClass('Measure').number - 1
            maqam = stream.Score()
            # Iterate through the measures in the original score and add them to the new score
            for measure in score.measures(start_measure_num, end_measure_num):
                maqam.insert(measure.offset, measure)

            maqam_list = maqam_info.get(element.content, [])
            maqam_list.append(extract_teslim_info(maqam))
            maqam_info[element.content] = maqam_list

            start_measure_num = end_measure_num + 1
            end_measure_num = None
        else:
            print('something went wrong')

    return maqam_info

In [37]:
maqam_info = find_maqams('../Makam List for One Key Signature.musicxml')
# {k: [d | {'accidentals': [a.nameWithOctave for a in d['accidentals']]} for d in v]for k, v in maqam_info.items()}

{'Pencgah': [{'start_note': <music21.note.Note D>,
   'end_note': <music21.note.Note G>,
   'range': (<music21.note.Note D>, <music21.note.Note G>),
   'accidentals': ['F#4', 'A#4', 'A4', 'C#5', 'C5', 'F5', 'F#5'],
   'all_pitches': [<music21.pitch.Pitch D4>,
    <music21.pitch.Pitch E4>,
    <music21.pitch.Pitch F#4>,
    <music21.pitch.Pitch G4>,
    <music21.pitch.Pitch A#4>,
    <music21.pitch.Pitch A4>,
    <music21.pitch.Pitch B4>,
    <music21.pitch.Pitch C#5>,
    <music21.pitch.Pitch C5>,
    <music21.pitch.Pitch D5>,
    <music21.pitch.Pitch E5>,
    <music21.pitch.Pitch F5>,
    <music21.pitch.Pitch F#5>,
    <music21.pitch.Pitch G5>]},
  {'start_note': <music21.note.Note A>,
   'end_note': <music21.note.Note A>,
   'range': (<music21.note.Note A>, <music21.note.Note A>),
   'accidentals': ['F5'],
   'all_pitches': [<music21.pitch.Pitch A4>,
    <music21.pitch.Pitch B4>,
    <music21.pitch.Pitch C5>,
    <music21.pitch.Pitch D5>,
    <music21.pitch.Pitch E5>,
    <music21.pi

In [45]:
folder_path = "../With Key Signatures/"

def get_common_pitch_info(teslim_pitches, maqam_pitches):
    common = []
    for teslim_pitch in teslim_pitches:
        if any(
            equal_pitch(teslim_pitch, maqam_pitch) for maqam_pitch in maqam_pitches
        ):
            common.append(teslim_pitch)
    teslim_only = []
    for teslim_pitch in teslim_pitches:
        if not any(
            equal_pitch(teslim_pitch, common_pitch) for common_pitch in common
        ):
            teslim_only.append(teslim_pitch)
    maqam_only = []
    for maqam_pitch in maqam_pitches:
        if not any(
            equal_pitch(maqam_pitch, common_pitch) for common_pitch in common
        ):
            maqam_only.append(maqam_pitch)
    return {
        'common': common,
        'teslim_only': teslim_only,
        'maqam_only': maqam_only
    }

for musicfile in os.listdir(folder_path):
    try:
        print(musicfile)
        teslim = find_teslim(musicfile)
        teslim_info = extract_teslim_info(teslim)
        for maqam_name, infos in maqam_info.items():
            for info in infos:
#                 common = teslim_info['all_pitches'] | info['all_pitches']
#                 teslim_only = teslim_info['all_pitches']- common
#                 maqam_only = info['all_pitches'] - common
                print(maqam_name)
                print({k: len(v) for k, v in get_common_pitch_info(teslim_info['all_pitches'], info['all_pitches']).items()})
#                 print(f'common: {len(common)} teslim only: {len(teslim_only)} maqam only: {len(maqam_only)}')
        print()
    except Exception as e:
        print(musicfile)
        print(e)


CT 315 Hicâz ‘Aşîrân Sâz Semâ’îsi. İsma’îl Ağa’nın.musicxml
Pencgah
{'common': 3, 'teslim_only': 0, 'maqam_only': 13}
Pencgah
{'common': 3, 'teslim_only': 0, 'maqam_only': 7}
Rehavi
{'common': 3, 'teslim_only': 0, 'maqam_only': 11}
Rehavi
{'common': 3, 'teslim_only': 0, 'maqam_only': 12}
Pesendide
{'common': 3, 'teslim_only': 0, 'maqam_only': 11}
Pesendide
{'common': 3, 'teslim_only': 0, 'maqam_only': 9}
Pesendide
{'common': 3, 'teslim_only': 0, 'maqam_only': 6}
Sazkar
{'common': 0, 'teslim_only': 3, 'maqam_only': 8}
Sazkar
{'common': 3, 'teslim_only': 0, 'maqam_only': 7}
Sazkar
{'common': 3, 'teslim_only': 0, 'maqam_only': 8}
Evc
{'common': 3, 'teslim_only': 0, 'maqam_only': 8}
Evc (more of a fragment that can descend down)
{'common': 0, 'teslim_only': 3, 'maqam_only': 11}
Neva
{'common': 3, 'teslim_only': 0, 'maqam_only': 3}
Tahir
{'common': 3, 'teslim_only': 0, 'maqam_only': 8}
Tahir Buselik
{'common': 3, 'teslim_only': 0, 'maqam_only': 13}
Tahir Buslik
{'common': 3, 'teslim_only': 

Pencgah
{'common': 0, 'teslim_only': 12, 'maqam_only': 14}
Pencgah
{'common': 0, 'teslim_only': 12, 'maqam_only': 8}
Rehavi
{'common': 0, 'teslim_only': 12, 'maqam_only': 12}
Rehavi
{'common': 0, 'teslim_only': 12, 'maqam_only': 13}
Pesendide
{'common': 0, 'teslim_only': 12, 'maqam_only': 12}
Pesendide
{'common': 0, 'teslim_only': 12, 'maqam_only': 10}
Pesendide
{'common': 0, 'teslim_only': 12, 'maqam_only': 7}
Sazkar
{'common': 0, 'teslim_only': 12, 'maqam_only': 8}
Sazkar
{'common': 0, 'teslim_only': 12, 'maqam_only': 8}
Sazkar
{'common': 0, 'teslim_only': 12, 'maqam_only': 9}
Evc
{'common': 0, 'teslim_only': 12, 'maqam_only': 9}
Evc (more of a fragment that can descend down)
{'common': 0, 'teslim_only': 12, 'maqam_only': 11}
Neva
{'common': 0, 'teslim_only': 12, 'maqam_only': 4}
Tahir
{'common': 0, 'teslim_only': 12, 'maqam_only': 9}
Tahir Buselik
{'common': 0, 'teslim_only': 12, 'maqam_only': 14}
Tahir Buslik
{'common': 0, 'teslim_only': 12, 'maqam_only': 10}
Huseyni
{'common': 0, 

Pencgah
{'common': 0, 'teslim_only': 6, 'maqam_only': 14}
Pencgah
{'common': 6, 'teslim_only': 0, 'maqam_only': 7}
Rehavi
{'common': 0, 'teslim_only': 6, 'maqam_only': 12}
Rehavi
{'common': 6, 'teslim_only': 0, 'maqam_only': 12}
Pesendide
{'common': 6, 'teslim_only': 0, 'maqam_only': 11}
Pesendide
{'common': 6, 'teslim_only': 0, 'maqam_only': 9}
Pesendide
{'common': 0, 'teslim_only': 6, 'maqam_only': 7}
Sazkar
{'common': 0, 'teslim_only': 6, 'maqam_only': 8}
Sazkar
{'common': 0, 'teslim_only': 6, 'maqam_only': 8}
Sazkar
{'common': 0, 'teslim_only': 6, 'maqam_only': 9}
Evc
{'common': 0, 'teslim_only': 6, 'maqam_only': 9}
Evc (more of a fragment that can descend down)
{'common': 6, 'teslim_only': 0, 'maqam_only': 10}
Neva
{'common': 6, 'teslim_only': 0, 'maqam_only': 3}
Tahir
{'common': 6, 'teslim_only': 0, 'maqam_only': 8}
Tahir Buselik
{'common': 6, 'teslim_only': 0, 'maqam_only': 13}
Tahir Buslik
{'common': 6, 'teslim_only': 0, 'maqam_only': 9}
Huseyni
{'common': 6, 'teslim_only': 0, 

Pencgah
{'common': 6, 'teslim_only': 0, 'maqam_only': 13}
Pencgah
{'common': 6, 'teslim_only': 0, 'maqam_only': 7}
Rehavi
{'common': 6, 'teslim_only': 0, 'maqam_only': 11}
Rehavi
{'common': 6, 'teslim_only': 0, 'maqam_only': 12}
Pesendide
{'common': 6, 'teslim_only': 0, 'maqam_only': 11}
Pesendide
{'common': 6, 'teslim_only': 0, 'maqam_only': 9}
Pesendide
{'common': 6, 'teslim_only': 0, 'maqam_only': 6}
Sazkar
{'common': 6, 'teslim_only': 0, 'maqam_only': 7}
Sazkar
{'common': 6, 'teslim_only': 0, 'maqam_only': 7}
Sazkar
{'common': 6, 'teslim_only': 0, 'maqam_only': 8}
Evc
{'common': 6, 'teslim_only': 0, 'maqam_only': 8}
Evc (more of a fragment that can descend down)
{'common': 6, 'teslim_only': 0, 'maqam_only': 10}
Neva
{'common': 0, 'teslim_only': 6, 'maqam_only': 4}
Tahir
{'common': 6, 'teslim_only': 0, 'maqam_only': 8}
Tahir Buselik
{'common': 6, 'teslim_only': 0, 'maqam_only': 13}
Tahir Buslik
{'common': 6, 'teslim_only': 0, 'maqam_only': 9}
Huseyni
{'common': 0, 'teslim_only': 6, 

Pencgah
{'common': 0, 'teslim_only': 12, 'maqam_only': 14}
Pencgah
{'common': 12, 'teslim_only': 0, 'maqam_only': 7}
Rehavi
{'common': 0, 'teslim_only': 12, 'maqam_only': 12}
Rehavi
{'common': 12, 'teslim_only': 0, 'maqam_only': 12}
Pesendide
{'common': 12, 'teslim_only': 0, 'maqam_only': 11}
Pesendide
{'common': 12, 'teslim_only': 0, 'maqam_only': 9}
Pesendide
{'common': 0, 'teslim_only': 12, 'maqam_only': 7}
Sazkar
{'common': 0, 'teslim_only': 12, 'maqam_only': 8}
Sazkar
{'common': 12, 'teslim_only': 0, 'maqam_only': 7}
Sazkar
{'common': 12, 'teslim_only': 0, 'maqam_only': 8}
Evc
{'common': 12, 'teslim_only': 0, 'maqam_only': 8}
Evc (more of a fragment that can descend down)
{'common': 0, 'teslim_only': 12, 'maqam_only': 11}
Neva
{'common': 0, 'teslim_only': 12, 'maqam_only': 4}
Tahir
{'common': 12, 'teslim_only': 0, 'maqam_only': 8}
Tahir Buselik
{'common': 12, 'teslim_only': 0, 'maqam_only': 13}
Tahir Buslik
{'common': 0, 'teslim_only': 12, 'maqam_only': 10}
Huseyni
{'common': 12, 

Pencgah
{'common': 0, 'teslim_only': 24, 'maqam_only': 14}
Pencgah
{'common': 0, 'teslim_only': 24, 'maqam_only': 8}
Rehavi
{'common': 0, 'teslim_only': 24, 'maqam_only': 12}
Rehavi
{'common': 0, 'teslim_only': 24, 'maqam_only': 13}
Pesendide
{'common': 0, 'teslim_only': 24, 'maqam_only': 12}
Pesendide
{'common': 0, 'teslim_only': 24, 'maqam_only': 10}
Pesendide
{'common': 0, 'teslim_only': 24, 'maqam_only': 7}
Sazkar
{'common': 0, 'teslim_only': 24, 'maqam_only': 8}
Sazkar
{'common': 0, 'teslim_only': 24, 'maqam_only': 8}
Sazkar
{'common': 0, 'teslim_only': 24, 'maqam_only': 9}
Evc
{'common': 0, 'teslim_only': 24, 'maqam_only': 9}
Evc (more of a fragment that can descend down)
{'common': 0, 'teslim_only': 24, 'maqam_only': 11}
Neva
{'common': 0, 'teslim_only': 24, 'maqam_only': 4}
Tahir
{'common': 0, 'teslim_only': 24, 'maqam_only': 9}
Tahir Buselik
{'common': 0, 'teslim_only': 24, 'maqam_only': 14}
Tahir Buslik
{'common': 0, 'teslim_only': 24, 'maqam_only': 10}
Huseyni
{'common': 0, 

Pencgah
{'common': 9, 'teslim_only': 0, 'maqam_only': 13}
Pencgah
{'common': 9, 'teslim_only': 0, 'maqam_only': 7}
Rehavi
{'common': 9, 'teslim_only': 0, 'maqam_only': 11}
Rehavi
{'common': 9, 'teslim_only': 0, 'maqam_only': 12}
Pesendide
{'common': 9, 'teslim_only': 0, 'maqam_only': 11}
Pesendide
{'common': 9, 'teslim_only': 0, 'maqam_only': 9}
Pesendide
{'common': 9, 'teslim_only': 0, 'maqam_only': 6}
Sazkar
{'common': 9, 'teslim_only': 0, 'maqam_only': 7}
Sazkar
{'common': 9, 'teslim_only': 0, 'maqam_only': 7}
Sazkar
{'common': 9, 'teslim_only': 0, 'maqam_only': 8}
Evc
{'common': 9, 'teslim_only': 0, 'maqam_only': 8}
Evc (more of a fragment that can descend down)
{'common': 9, 'teslim_only': 0, 'maqam_only': 10}
Neva
{'common': 0, 'teslim_only': 9, 'maqam_only': 4}
Tahir
{'common': 9, 'teslim_only': 0, 'maqam_only': 8}
Tahir Buselik
{'common': 9, 'teslim_only': 0, 'maqam_only': 13}
Tahir Buslik
{'common': 9, 'teslim_only': 0, 'maqam_only': 9}
Huseyni
{'common': 9, 'teslim_only': 0, 

Pencgah
{'common': 10, 'teslim_only': 0, 'maqam_only': 13}
Pencgah
{'common': 0, 'teslim_only': 10, 'maqam_only': 8}
Rehavi
{'common': 0, 'teslim_only': 10, 'maqam_only': 12}
Rehavi
{'common': 0, 'teslim_only': 10, 'maqam_only': 13}
Pesendide
{'common': 10, 'teslim_only': 0, 'maqam_only': 11}
Pesendide
{'common': 0, 'teslim_only': 10, 'maqam_only': 10}
Pesendide
{'common': 0, 'teslim_only': 10, 'maqam_only': 7}
Sazkar
{'common': 0, 'teslim_only': 10, 'maqam_only': 8}
Sazkar
{'common': 0, 'teslim_only': 10, 'maqam_only': 8}
Sazkar
{'common': 0, 'teslim_only': 10, 'maqam_only': 9}
Evc
{'common': 0, 'teslim_only': 10, 'maqam_only': 9}
Evc (more of a fragment that can descend down)
{'common': 0, 'teslim_only': 10, 'maqam_only': 11}
Neva
{'common': 10, 'teslim_only': 0, 'maqam_only': 3}
Tahir
{'common': 0, 'teslim_only': 10, 'maqam_only': 9}
Tahir Buselik
{'common': 0, 'teslim_only': 10, 'maqam_only': 14}
Tahir Buslik
{'common': 0, 'teslim_only': 10, 'maqam_only': 10}
Huseyni
{'common': 0, 

Pencgah
{'common': 1, 'teslim_only': 0, 'maqam_only': 13}
Pencgah
{'common': 1, 'teslim_only': 0, 'maqam_only': 7}
Rehavi
{'common': 1, 'teslim_only': 0, 'maqam_only': 11}
Rehavi
{'common': 1, 'teslim_only': 0, 'maqam_only': 12}
Pesendide
{'common': 1, 'teslim_only': 0, 'maqam_only': 11}
Pesendide
{'common': 0, 'teslim_only': 1, 'maqam_only': 10}
Pesendide
{'common': 1, 'teslim_only': 0, 'maqam_only': 6}
Sazkar
{'common': 1, 'teslim_only': 0, 'maqam_only': 7}
Sazkar
{'common': 0, 'teslim_only': 1, 'maqam_only': 8}
Sazkar
{'common': 1, 'teslim_only': 0, 'maqam_only': 8}
Evc
{'common': 0, 'teslim_only': 1, 'maqam_only': 9}
Evc (more of a fragment that can descend down)
{'common': 1, 'teslim_only': 0, 'maqam_only': 10}
Neva
{'common': 0, 'teslim_only': 1, 'maqam_only': 4}
Tahir
{'common': 1, 'teslim_only': 0, 'maqam_only': 8}
Tahir Buselik
{'common': 1, 'teslim_only': 0, 'maqam_only': 13}
Tahir Buslik
{'common': 1, 'teslim_only': 0, 'maqam_only': 9}
Huseyni
{'common': 0, 'teslim_only': 1,

Pencgah
{'common': 11, 'teslim_only': 0, 'maqam_only': 13}
Pencgah
{'common': 11, 'teslim_only': 0, 'maqam_only': 7}
Rehavi
{'common': 11, 'teslim_only': 0, 'maqam_only': 11}
Rehavi
{'common': 11, 'teslim_only': 0, 'maqam_only': 12}
Pesendide
{'common': 11, 'teslim_only': 0, 'maqam_only': 11}
Pesendide
{'common': 11, 'teslim_only': 0, 'maqam_only': 9}
Pesendide
{'common': 11, 'teslim_only': 0, 'maqam_only': 6}
Sazkar
{'common': 11, 'teslim_only': 0, 'maqam_only': 7}
Sazkar
{'common': 11, 'teslim_only': 0, 'maqam_only': 7}
Sazkar
{'common': 11, 'teslim_only': 0, 'maqam_only': 8}
Evc
{'common': 11, 'teslim_only': 0, 'maqam_only': 8}
Evc (more of a fragment that can descend down)
{'common': 11, 'teslim_only': 0, 'maqam_only': 10}
Neva
{'common': 0, 'teslim_only': 11, 'maqam_only': 4}
Tahir
{'common': 11, 'teslim_only': 0, 'maqam_only': 8}
Tahir Buselik
{'common': 11, 'teslim_only': 0, 'maqam_only': 13}
Tahir Buslik
{'common': 11, 'teslim_only': 0, 'maqam_only': 9}
Huseyni
{'common': 11, '